# Assignment 5 - Matrix Decomposition Method

## Overview
1. Implement matrix decomposition method to decompose matrices
- Method of decomposition: Doolittle, Crout, Cholesky, $LDL^{T}$, and Singular Value Decomposition (SVD)
2. Solve linear equation via decomposed matrices to see the improvement in performance

In [1]:
import numpy as np

## Q1

In [2]:
M1 = np.array([[4, 0, 0, 0],
               [6, 7, 0, 0],
               [9, 11, 1, 0],
               [5, 4, 1, 1]])

In [3]:
M2 = np.array([[2, 3, 1, 2],
               [-2, 4, -1, 5],
               [3, 7, 1.5, 1],
               [6, -9, 3, 7]])

#### a) Check Symmetry

In [4]:
def if_symmetric(A):
    return (A == (A.T)).all()

In [5]:
if_symmetric(M1)

False

In [6]:
if_symmetric(M2)

False

Both matrices are not symmetric

#### b) Check Singularity

In [7]:
def if_singular(A):
    return np.linalg.det(A) == 0

In [8]:
if_singular(M1)

False

In [9]:
if_singular(M2)

False

Both matrices are not singular

#### c) Check Positive Definite

In [10]:
def if_posdef(A):
    
    n = A.shape[0]
    #(1) Check symmetry
    if (A != A.T).any():
        print('The matrix is not symmetric')
        return False
    
    #Leading principle submatrix of A
    for k in range(1, n+1):
        if np.linalg.det(A[:k, :k]) <= 0:
            print('Determinant of the leading principle submatrix is not positive')
            return False
    return True        

In [11]:
if_posdef(M1)

The matrix is not symmetric


False

In [12]:
if_posdef(M2)

The matrix is not symmetric


False

Both matrices are not positive definite

---

## Q2 - Matrix Factorization

### 1. Factorization

#### 1.1 $LDL^T$ Factorization

In [13]:
def LDLt(A):
    """
    Arguments:
    - A [array] to be factorized, shape (n, n)
    Return:
    - L [array] Lower triangular matrix of shape (n,n)
    - D [array] Diagonal matrix of shape (n,n)
    """
    
    n = A.shape[0]
    
    #Initialize L and D
    L = np.zeros((n,n))
    D = np.zeros((n,n))
    d = np.zeros(n)
    
    #Diagonal of L
    np.fill_diagonal(L, 1)
    
    for i in range(n):
        #(1)
        sumLij_wj = 0
        for j in range(i):
            wj = L[i, j]*d[j]
            sumLij_wj += L[i,j]*wj
        
        #(2) Get diagonal entries of D
        d[i] = A[i,i] - sumLij_wj
        
        #(3) Get entries of L (not in diagonal)
        
        for j in range(i+1, n):
            sumLjk_wk = 0
            for k in range(i):
                wk = L[j-1, k]*d[k]
                sumLjk_wk += L[j, k] * wk
            
            L[j, i] = (A[j, i] - sumLjk_wk)/d[i]
    
    np.fill_diagonal(D, d)
    
    return L, D
    

#### 1.2. Cholesky Factorization

In [14]:
def Cholesky(A):
    
    if if_posdef(A) == False:
        raise Exception("Input matrix must be positive definite")
    
    n = A.shape[0]
    
    #Initialize L
    L = np.zeros((n,n))
    
    #(1) l11
    L[0, 0] = np.sqrt(A[0, 0])
    
    #(2) First column of L
    for j in range(1, n):
        L[j, 0] = A[j, 0]/L[0,0]
        
    for i in range(n-1):
        #(3.1) Diagonal entries of L
        sumLik_sq = 0
        for k in range(i):
            sumLik_sq += L[i, k]**2
        L[i,i] = np.sqrt((A[i,i] - sumLik_sq))
        
        #(3.2)
        for j in range(i+1, n):
            sumLjk_Lik = 0
            for k in range(i):
                sumLjk_Lik += L[j, k]*L[i,k]
            L[j, i] = (A[j, i] - sumLjk_Lik)/L[i, i]
        
    
    #(4) Last entry
    sumLnk_sq = 0
    for k in range(n-1):
        sumLnk_sq += L[-1, k]**2
    L[-1, -1] = np.sqrt(A[-1, -1] - sumLnk_sq)
    
    return L

#### 1.3. LU Factorization

In [15]:
def LU(A):
    """
    Arguments:
    - A [array] to be factorized, shape (n, n)
    Return:
    - L [array] Lower triangular matrix of the same shape as A
    - U [array] Upper triangular matrix of the same shape as A
    """
    
    n = A.shape[0]
    
    #Initialize L and U as matrix of 0's
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    #Set the diagonal entries of L as 1's
    np.fill_diagonal(L, 1)
    
    #(1) Set u[0,0]
    if A[0,0] == 0:
        raise Exception("Impossible factorization")
    
    #(2) 1st row of U and 1st column of L
    U[0,:] = A[0,:]/L[0,0]
    L[:,0] = A[:,0]/U[0,0]
    
    #(3)
    for i in range(1, n-1):
        
        #(3.1) Diagonal entries of U
        sumLik_Uki = 0
        for k in range(i):
            sumLik_Uki += L[i, k]*U[k, i]
        U[i,i] = 1/L[i,i] * (A[i,i] - sumLik_Uki)
        
        if U[i,i] == 0:
            raise Exception("Impossible factorization")
        
        #(3.2) Other entries of U and L within the borders and not in diagonal
        for j in range(i+1, n):
            sumLik_Ukj = 0
            sumLjk_Uki = 0
            for k in range(i):
                sumLik_Ukj += L[i, k]*U[k, j]
                sumLjk_Uki += L[j, k]*U[k, i]
            U[i, j] = 1/L[i,i] * (A[i, j] - sumLik_Ukj)
            L[j, i] = 1/U[i,i] * (A[j, i] - sumLjk_Uki)
        
        #(4) U[n-1, n-1]
        Ln1k_Ukn1 = np.array([])
        for k in range(n-1):
            Ln1k_Ukn1 = np.append(Ln1k_Ukn1, L[n-1, k]*U[k, n-1])
        U[n-1, n-1] = 1/L[n-1, n-1] * (A[n-1, n-1] - Ln1k_Ukn1.sum())
        
    return L, U    

#### 1.4. Crout Factorization

In [16]:
def Crout(A, solve = False):
    
    if solve:
        if A.shape[0] == A.shape[1]:
            raise Exception("The input matrix must be of shape (n, n+1) to be solved")
    
    n = A.shape[0]
    b = A[:, n]
    A = A[:, :n]
    
    #------ Factorize --------
    #Initialize
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    
    #Set diagonal entries of U
    np.fill_diagonal(U, 1)
    
    #(1) First element of L
    L[0, 0] = A[0,0]
    
    #(2) First row of U
    U[0, :] = A[0, :] / L[0,0]
    
    #(3) First column of L
    L[:,0] = A[:, 0]
    
    
    for i in range(1, n):
        
        #(4) Diagonal entries of L
        sumLik_Uki = 0
        for k in range(i):
            sumLik_Uki += L[i, k]*U[k,i]
            
        L[i, i] = A[i, i] - sumLik_Uki
        if (L[i, i] == 0):
            raise Exception('Impossible factorization')
        
        #(5) Find entries above diagonal of U and below diagonal of L
        for j in range(i+1, n):
            sumLik_Ukj = 0
            sumLjk_Uki = 0
            for k in range(i):
                sumLik_Ukj += L[i, k]*U[k, j]
                sumLjk_Uki += L[j, k]*U[k, i]
                
            #(5.1) Entries above diagonal of U    
            U[i, j] = 1/L[i,i] * (A[i, j] - sumLik_Ukj)
            
            #(5.2) Entries below diagonal of L
            L[j, i] = 1/U[i,i] * (A[j, i] - sumLjk_Uki)
        
    #(6) The last entriy of L
    sumLnk_Ukn = 0
    for k in range(n-1):
        sumLnk_Ukn += L[-1, k]*U[k, -1]
    L[-1, -1] = A[-1, -1] - sumLnk_Ukn
    
    
    #------Solve for X----------
    if solve:
        Z = np.zeros(n)
        X = np.zeros(n)
        
        #(1) Find Z
        Z[0] = b[0]/L[0,0]
        
        for i in range(1, n):
            sumLij_Zj = 0
            for j in range(i):
                sumLij_Zj += L[i, j]*Z[j]
            Z[i] = 1/L[i,i] * (b[i] - sumLij_Zj)
        
        
        #(2) Find X
        X[-1] = Z[-1]
        
        for i in range(n-1, -1, -1):
            sumUij_Xj = 0
            for j in range(i+1, n):
                sumUij_Xj += U[i, j]*X[j]
            X[i] = (Z[i] - sumUij_Xj)
            #X[i] = Z[i] - U[i, i+1]*X[i+1]
        
        return X
        
    return L, U

#### Check functions

In [17]:
C = np.array([[4, 1, 1, 1],
              [1, 3, -1, 1],
              [1, -1, 2, 0],
              [1, 1, 0, 2]])

LC, DC = LDLt(C)

In [18]:
check_C = np.dot( np.dot(LC, DC), LC.T )
np.isclose(check_C, C)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

The LDLt fatorization produce the correct component matrices of A 

In [19]:
LCc = Cholesky(C)

np.isclose(C,
           np.dot(LCc, LCc.T))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

__Comment:__ The `Cholesky` factorization produces correct component matrices

### 2. Solve systems of equations

#### 2.1 Solve by $LU$ factorization

In [20]:
A = np.array([[1, 1, 0, 3],
              [2, 1, -1, 1],
              [3, -1, -1, 2],
              [-1, 2, 3, -1]])
b = np.array([8, 7, 14, -7])

L, U = LU(A)

In [21]:
n = A.shape[0]

y = np.zeros(b.shape)
X = np.zeros(b.shape)

#Solve Ly = b
y[0] = b[0]/L[0,0]

for i in range(1, n):
    sumLij_yj = 0
    for j in range(i):
        sumLij_yj += L[i, j]*y[j]
    
    y[i] = 1/L[i,i] * (b[i] - sumLij_yj)
    

#Solve UX = y
X[-1] = y[-1]/U[-1,-1]

for i in range(n-1, -1, -1):
    sumUij_Xj = 0
    for j in range(i+1, n):
        sumUij_Xj += U[i, j]*X[j]
    X[i] = 1/U[i,i] * (y[i] - sumUij_Xj)
    

In [22]:
np.dot(A, X) - b

array([0., 0., 0., 0.])

In [23]:
X

array([ 3., -1.,  0.,  2.])

#### 2.2 Solve by Crout method

In [24]:
A = np.array([[2, 1, 0, 3],
              [1, 2, 1, -2],
              [0, 2, 3, 0]])

In [25]:
Lcrout, Ucrout = Crout(A, solve = False)

In [26]:
#Solve for X by Crout method
Xcrout = Crout(A, solve = True)
Xcrout

array([ 3.6, -4.2,  2.8])

In [27]:
#Plug-in to check if X is correct
np.dot(A[:, :3], Xcrout) - A[:, -1]

array([0., 0., 0.])

In [28]:
#Check decomposition
np.dot(Lcrout, Ucrout) - A[:, :3]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

---

## Q3 - SVD 

In [29]:
def SVD(A):
    
    m = A.shape[0]
    n = A.shape[1]
    
    # (1) Find eigenvalues and normalized eigenvectors V
    eigvl, V = np.linalg.eig(np.dot(A.T, A))
    
    #(2) Find S

    k = len(eigvl)
    S = np.zeros((m, n))

    for i in range(k):
        S[i,i] = np.sqrt(eigvl[i])
    
    #(3) Find U
    U = np.zeros((m, m)) 

    for i in range(k):
        U[:,i] = 1/S[i, i] * np.dot(A, V[:, i])
    
    return U, S, V.T

In [30]:
A = np.array([[1, 0, 1],
              [0, 1, 0],
              [0, 1, 1],
              [0, 1, 0],
              [1, 1, 0]])

In [31]:
U, S, V_transpose = SVD(A)

In [32]:
V_transpose

array([[-4.08248290e-01, -8.16496581e-01, -4.08248290e-01],
       [-7.07106781e-01,  5.12349117e-16,  7.07106781e-01],
       [ 5.77350269e-01, -5.77350269e-01,  5.77350269e-01]])

In [33]:
U

array([[-3.65148372e-01, -1.11022302e-16,  8.16496581e-01,
         0.00000000e+00,  0.00000000e+00],
       [-3.65148372e-01,  5.12349117e-16, -4.08248290e-01,
         0.00000000e+00,  0.00000000e+00],
       [-5.47722558e-01,  7.07106781e-01, -1.57009246e-16,
         0.00000000e+00,  0.00000000e+00],
       [-3.65148372e-01,  5.12349117e-16, -4.08248290e-01,
         0.00000000e+00,  0.00000000e+00],
       [-5.47722558e-01, -7.07106781e-01, -3.92523115e-16,
         0.00000000e+00,  0.00000000e+00]])

In [34]:
S

array([[2.23606798, 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.41421356],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])